In [3]:
using DataFrames              # package for working with data sets
using JWAS                    # package for Bayesian regression analyses, including BayesB and BayesCπ        
using JWAS.Datasets
using Distributions       
using Plots                   # package for plotting 
using CSV
using LinearAlgebra,Statistics,Random,DelimitedFiles, DataFrames

## BV fomr MHC region from MCMC BayesB samples

In [14]:
SNP_Effect_ADG= readdlm("MCMC_samples_marker_effects_Nur2ADG.txt", ',',header=false)
SNP_Effect_ADG = SNP_Effect_ADG[2:451,:]

451×435172 Array{Float64,2}:
  1.0          2.0  3.0  4.0   5.0          …  435171.0          435172.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  0.0          0.0  0.0  0.0   0.0          …       0.0               0.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  0.0          0.0  0.0  0.0   0.0                 -0.000583785       0.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  0.0          0.0  0.0  0.0   0.0          …       0.0               0.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  0.0          0.0  0.0  0.0   0.0                  0.0               0.0
  ⋮      

In [16]:
# Get marker effects for MHC region 20 to 26mb
Effect_ADG_MHC= SNP_Effect_ADG[:, 185212:186439]  # obtained from Map info

450×1228 Array{Float64,2}:
 0.0  0.0          0.0  0.0         0.0  …  0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0  -0.001191   
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0  -0.000155688
 0.0  0.0          0.0  0.00195145  0.0  …  0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0  …  0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  0.0  0.0  0.0   0.0        
 0.0  0.0          0.0  0.0         0.0     0.0  

In [17]:
function readMatBin(fileName)
    genStr = open(fileName)
    n = read(genStr,Int64)
    p = read(genStr,Int64)
    M = zeros(n,p)
    for j in 1:p
        for i in 1:n
            M[i,j] = read(genStr,Float64)
        end
    end
    close(genStr)
    return M
end
@time M = readMatBin("/home/jovyan/mark/JWAS_CDPQ/CDPQ_all_data/ALGP2_Cycle_1_7_Geno_Complete_Impute_JWAS.bin.gen.bin")

 18.741101 seconds (30.84 k allocations: 10.393 GiB, 0.29% gc time)


3205×435172 Array{Float64,2}:
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  2.0  2.0  0.0  1.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  1.0  2.0  0.0  1.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  2.0  2.0  0.0  2.0  2.0     0.0  2.0  1.0  2.0  2.0  2.0  2.0
 1.0  1.0  0.0  1.0  2.0  0.0  0.0  2.0  …  0.0  2.0  0.0  2.0  2.0  2.0  2.0
 0.0  2.0  0.0  0.0  2.0  0.0  0.0  2.0     0.0  2.0  0.0  2.0  2.0  2.0  2.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [18]:
# Get genotypes for MHC region
M_MHC= M[:, 185212:186439]
M_MHC'

1228×3205 Adjoint{Float64,Array{Float64,2}}:
 1.0  1.0  1.0  2.0  1.0  1.0  1.0  2.0  …  1.0  1.0  2.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  2.0  1.0  1.0  1.0  2.0     2.0  1.0  2.0  1.0  1.0  1.0  1.0
 1.0  1.0  0.0  2.0  1.0  1.0  1.0  2.0     1.0  1.0  2.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0     0.0  1.0  0.0  1.0  1.0  1.0  1.0
 1.0  1.0  2.0  0.0  1.0  1.0  1.0  0.0     1.0  1.0  0.0  1.0  1.0  1.0  2.0
 1.0  1.0  1.0  2.0  1.0  1.0  1.0  2.0  …  1.0  1.0  2.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  2.0  1.0  1.0  1.0  2.0     2.0  1.0  2.0  1.0  1.0  1.0  1.0
 2.0  2.0  1.0  2.0  2.0  1.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  1.0
 1.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0     0.0  1.0  0.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0     1.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  2.0  1.0  0.0  1.0  2.0  …  1.0  2.0  1.0  2.0  2.0  2.0  2.0
 2.0  2.0  1.0  2.0  2.0  2.0  2.0  2.0     1.0  2.0  2.0  2.0  2.0  2.0  2.0
 1.0  1.0  2.0  2.0

In [19]:
BV_ADG_MHC= Effect_ADG_MHC * M_MHC'

450×3205 Array{Float64,2}:
 -0.00336865  -0.0393616   -0.0563197   …  -0.0359425   -0.00334134
 -0.0158482   -0.0345365   -0.0331267      -0.0334873   -0.0153753 
 -0.0275064   -0.0566526   -0.0626319      -0.0581492   -0.0296815 
 -0.0140185   -0.0336538   -0.0376968      -0.0337436   -0.0165067 
 -0.0217891   -0.041731    -0.0367506      -0.0430378   -0.0184747 
 -0.0226796   -0.0414795   -0.0407245   …  -0.0455429   -0.0168697 
  0.00922567   0.0145289    0.0142954       0.00615519   0.00971636
  0.0557695    0.0249465   -0.00374        -0.00294369   0.0256897 
  0.0375299    0.017261    -0.0117972      -0.00407811   0.0151704 
  0.0193799    0.00791149  -0.0077901      -0.00904564   0.0110858 
  0.0619656    0.042816     0.0106806   …   0.0109227    0.0320367 
  0.0556903    0.0279543   -0.00133116      0.00341958   0.0276335 
  0.0836214    0.0490203    0.0164471       0.0146392    0.0546081 
  ⋮                                     ⋱                          
 -0.0117662   -0.0341

In [31]:
BV= vec(mean(BV_ADG_MHC, dims = 1))

In [36]:
[round(i,digits=4) for i in BV]

3205-element Array{Float64,1}:
 -0.0056
 -0.0245
 -0.0294
 -0.0274
 -0.0255
 -0.0134
 -0.0056
 -0.0273
 -0.0301
 -0.0255
 -0.0247
 -0.0294
 -0.0287
  ⋮     
 -0.0068
 -0.0278
  0.0041
 -0.0163
 -0.0292
  0.0078
 -0.0122
 -0.0308
 -0.0093
 -0.0252
 -0.0296
 -0.0104

In [38]:
writedlm("MHC_BV.txt", BV)

## Model frequency

In [41]:
modelFreq= readdlm("marker_effects_BayesB.txt", ',',header=false)

435173×5 Array{Any,2}:
 "Trait"          "Marker_ID"    "Estimate"   "Std_Error"   "Model_Frequency"
 "Nur2ADG"       1              1.20295e-5   0.000192102   0.0288889         
 "Nur2ADG"       2              3.899e-6     0.000138743   0.02              
 "Nur2ADG"       3              8.98364e-6   0.000111313   0.00888889        
 "Nur2ADG"       4              3.87155e-6   0.000130085   0.0155556         
 "Nur2ADG"       5             -3.14116e-6   4.19465e-5    0.0133333         
 "Nur2ADG"       6             -1.53562e-6   6.56473e-5    0.00888889        
 "Nur2ADG"       7             -1.2385e-6    0.000174501   0.0177778         
 "Nur2ADG"       8              2.05404e-6   9.93692e-5    0.0155556         
 "Nur2ADG"       9             -9.04286e-7   7.04165e-5    0.0155556         
 "Nur2ADG"      10             -5.16968e-6   9.89559e-5    0.02              
 "Nur2ADG"      11              3.28748e-6   8.80124e-5    0.00888889        
 "Nur2ADG"      12              8.80986e-